In [ ]:
"""
This script uses mm2python to stream data to napari, as well as processing
"""

In [ ]:

%gui qt5
from py4j.java_gateway import JavaGateway
from napari import ViewerApp
import cv2
import time
import numpy as np
from PyQt5.QtCore import QThreadPool, QRunnable
import warnings
warnings.filterwarnings('ignore')


In [3]:
gate = JavaGateway()
viewer = ViewerApp()


In [4]:
layer1 = viewer.add_image(np.memmap('C:\\Users\\LabelFree\\Desktop\\nonramtemp\\Snap-Live-Stream.dat',
                                    dtype=np.uint16, offset=0,
                                    mode='r',
                                    shape=(2048, 2048)))




OSError: [WinError 8] Not enough storage is available to process this command

In [4]:
class ProcessRunnable(QRunnable):
    def __init__(self, target, args):
        QRunnable.__init__(self)
        self.t = target
        self.args = args

    def run(self):
        self.t(*self.args)

    def start(self):
        QThreadPool.globalInstance().start(self)


In [5]:
def stream_data():
    count = 0
    while True:
        time.sleep(0.001)
        layer1.image = np.memmap(
            'C:\\Users\\LabelFree\\Desktop\\nonramtemp\\Snap-Live-Stream.dat',
            dtype=np.uint16, offset=0,
            mode='r',
            shape=(2048, 2048))
        if count >= 10000:
            break
        count += 1


In [6]:
r = ProcessRunnable(target=stream_data, args=())
r.start()


In [ ]:
# demonstrate fast processing
viewer = ViewerApp()

In [11]:
def cv2_sobel_edge_with_binary(image):

    k=3
    t=5000
    blur = cv2.GaussianBlur(image, (k,k),0)

    (t, binary) = cv2.threshold(blur,t, 65534, cv2.THRESH_BINARY_INV)

    grad_x = cv2.Sobel(binary, cv2.CV_16U, 2, 0)
    grad_y = cv2.Sobel(binary, cv2.CV_16U, 0, 2)

    edge = cv2.bitwise_or(grad_x, grad_y)
    return edge


def stream_sobel():
    count = 0
    while True:
        time.sleep(0.001)
        layer1.image = cv2_sobel_edge_with_binary(
            np.memmap('Q:\Snap-Live-Stream.dat',
                      dtype=np.uint16, offset=0,
                      shape=(2048, 2048))
        )
        if count >= 10000:
            break
        count += 1


In [12]:
r = ProcessRunnable(target=stream_sobel, args=())
r.start()


In [ ]:
# demonstrate that we don't need a RAM disk

viewer = ViewerApp()

In [ ]:
layer1 = viewer.add_image(
    np.memmap('C:\\Users\\LabelFree\\Desktop\\nonramtemp\\Snap-Live-Stream.dat',
              dtype=np.uint16, offset=0,
              mode='r',
              shape=(2048, 2048)))


In [ ]:
def stream_data_nonram():
    count = 0
    while True:
        time.sleep(0.001)
        layer1.image = np.memmap(
            'C:\\Users\\LabelFree\\Desktop\\nonramtemp\\Snap-Live-Stream.dat',
            dtype=np.uint16, offset=0,
            mode='r',
            shape=(2048, 2048))
        if count >= 10000:
            break
        count += 1


In [ ]:
r = ProcessRunnable(target=stream_data, args=())
r.start()